In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv', index_col=0)
df.head()

In [ ]:
df.info()

In [ ]:
df[['cat'+str(i) for i in range(19)]] = df[['cat'+str(i) for i in range(19)]].astype('category')
df.info()

In [ ]:
df['target'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv', index_col=0)
df_test.head()

In [ ]:
df_test[['cat'+str(i) for i in range(19)]] = df_test[['cat'+str(i) for i in range(19)]].astype('category')
df_test.info()

In [ ]:
# Проверяем распределение категориальных признаков в train и test
for i in range(19):
    A = set(df['cat'+str(i)])
    B = set(df_test['cat'+str(i)])
    if B - A != set():
        print('Feature', i, 'has NEW values in df_test!!!')

In [ ]:
df_1 = df.drop('cat10', axis=1)
df_test_1 = df_test.drop('cat10', axis=1)

In [ ]:
fig, ax = plt.subplots(1, 2)
df['cat0'].value_counts().plot(kind='bar', ax=ax[0])
df_test['cat0'].value_counts().plot(kind='bar', ax=ax[1])
plt.show()

In [ ]:
for i in range(19):
    fig, ax = plt.subplots(1, 2)
    df['cat'+str(i)].value_counts().plot(kind='bar', ax=ax[0])
    df_test['cat'+str(i)].value_counts().plot(kind='bar', ax=ax[1])
    plt.show()

In [ ]:
df_2 = pd.get_dummies(df_1, columns=['cat'+str(i) for i in range(19) if i != 10], drop_first=True)
df_2.head()

In [ ]:
df_test_2 = pd.get_dummies(df_test_1, columns=['cat'+str(i) for i in range(19) if i != 10], 
                           drop_first=True)
df_test_2.head()

In [ ]:
X = df_2.drop('target', axis=1)
y = df_2['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=0.1, solver='saga')

# from sklearn.model_selection import GridSearchCV
# params = {'C': np.logspace(-3, 3, 7)}
# print(params)

In [ ]:
# model = GridSearchCV(logreg, params, cv=5, n_jobs=-1, scoring='roc_auc')
# model.fit(X_train, y_train)

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, logreg.predict_proba(X_test)[:, 1]))

In [ ]:
logreg.fit(X, y)

In [ ]:
subm = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')
subm.head()

In [ ]:
results = logreg.predict_proba(df_test_2)[:, 1]
df_results = pd.DataFrame(results, index=subm['id'], columns=['target'])
df_results.to_csv('my_submission.csv')

# Варианты улучшения

In [ ]:
# Random Forest
# Масштабирование
# Сжатие (PCA)
# Как-то поработать с фичей 10

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

print(roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1]))

In [ ]:
rf.fit(X, y)

In [ ]:
results = rf.predict_proba(df_test_2)[:, 1]
df_results = pd.DataFrame(results, index=subm['id'], columns=['target'])
df_results.to_csv('my_submission_rf.csv')